# Exploratory Guide for Long-Short Term Memory Neural Networks (LSTM)

This guide recreates the LSTM for Time Series article written by Jason Brownlee for machinelearningmastery.com

In [1]:
%cd ..
%cd .. 
%cd data

/Users/azel/github/Data-Science/jupyter_notebooks
/Users/azel/github/Data-Science
/Users/azel/github/Data-Science/data


In [95]:
from fin_data import DailyTimeSeries
import numpy as np
import pandas as pd
import os, sys

from sklearn.preprocessing import MinMaxScaler

    
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [104]:
def split(cut, X, y):
    """
    Temporal Train Test Split. Holdout is most recent data.  
    """
    
    # Train Test Split: 70% Train, 30% Test
    X_train = X[:int(len(X)*cut)]
    X_test= X[int(len(X)*cut):len(X)]
    y_train=y[:int(len(y)*cut)]
    y_test=y[int(len(y)*cut):len(y)]
    
    return X_train, X_test, y_train, y_test
    

### Univariate LSTM Model

In [4]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    
    for i in range (len(sequence)):
        #find the end of this pattern
        end_ix = i + n_steps
        # check to see if we are at the end of the sequence
        if end_ix > len(sequence)-1:
                break
        #gather input and output parts of the pattern
        seq_X, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_X)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [5]:
raw_seq = np.arange(start=10, stop=200, step=10)
n_steps = 5

X, y = split_sequence(raw_seq, n_steps)

for i in range(len(X)):
    print(X[i], y[i])

[10 20 30 40 50] 60
[20 30 40 50 60] 70
[30 40 50 60 70] 80
[40 50 60 70 80] 90
[50 60 70 80 90] 100
[ 60  70  80  90 100] 110
[ 70  80  90 100 110] 120
[ 80  90 100 110 120] 130
[ 90 100 110 120 130] 140
[100 110 120 130 140] 150
[110 120 130 140 150] 160
[120 130 140 150 160] 170
[130 140 150 160 170] 180
[140 150 160 170 180] 190


In [6]:
#from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [7]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [8]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=200, verbose=0)





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [9]:
#Creating next input
x_input = np.array([150, 160, 170,180,190])
#reshaping to the proper sequence
x_input = x_input.reshape((1, n_steps, n_features))

In [10]:
y_pred = model.predict(x_input, verbose=0)
print('The Prediction should be 200, is it?')
print('prediction: ', y_pred[0][0])

The Prediction should be 200, is it?
prediction:  198.98883


Now, let's apply this to the S&P500

In [11]:
#Get data for Apple
apple = DailyTimeSeries('AAPL')
adf = apple.initiate()
#Create Univariate Time Series 
data = adf[['AAPL close']]

################################################################### 
 Ticker:  AAPL 
 Last Refreshed:  2019-09-03 11:03:23 
 Data Retrieved:  Daily Prices (open, high, low, close) and Volumes 
 ###################################################################


In [12]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


values = [x for x in range(10)]
d = series_to_supervised(values)
d

,var1(t-1),var1(t)
1,0.0,1
2,1.0,2
3,2.0,3
4,3.0,4
5,4.0,5
6,5.0,6
7,6.0,7
8,7.0,8
9,8.0,9


In [13]:
data = series_to_supervised(data=data, n_in=3)

In [14]:
X = data.drop(columns='var1(t)')
y = data['var1(t)']

In [15]:
X_train, X_test, y_train, y_test = split(cut=0.8, X=X, y=y)

In [16]:
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [17]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, 
          epochs=200,
          validation_data=(X_test, y_test),
          verbose=0)

scores = model.evaluate(X_test, y_test)
print('Neural Network MSE: ', scores)

1007/1007 [==============================] - 0s 24us/step
Neural Network MSE:  8.19445793583708


### Stacked LSTM

Multiple hidden LSTM layers can be stacked one on top of another in what is referred to as a Stacked LSTM model.

An LSTM layer requires a three-dimensional input and LSTMs by default will produce a two-dimensional output as an interpretation from the end of the sequence.

We can address this by having the LSTM output a value for each time step in the input data by setting the return_sequences=True argument on the layer. This allows us to have 3D output from hidden LSTM layer as input to the next.

We can therefore define a Stacked LSTM as follows.

In [18]:
model= Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(50, activation='relu'))
model.add(Dense(50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, 
          epochs=200,
          validation_data=(X_test, y_test),
          verbose=0)

scores = model.evaluate(X_test, y_test)
print('Neural Network MSE: ', scores)

1007/1007 [==============================] - 0s 29us/step
Neural Network MSE:  9.247217992803428


### Bidirectional LSTM

On some sequence prediction problems, it can be beneficial to allow the LSTM model to learn the input sequence both forward and backwards and concatenate both interpretations.

This is called a Bidirectional LSTM.

We can implement a Bidirectional LSTM for univariate time series forecasting by wrapping the first hidden layer in a wrapper layer called Bidirectional.

An example of defining a Bidirectional LSTM to read input both forward and backward is as follows.

In [19]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, 
          epochs=200,
          validation_data=(X_test, y_test),
          verbose=0)

scores = model.evaluate(X_test, y_test)
print('Neural Network MSE: ', scores)

1007/1007 [==============================] - 0s 23us/step
Neural Network MSE:  7.887479213403969


### Multivariate LSTM Model

In [20]:
#Create Input
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
#Create target
out_seq

array([ 25,  45,  65,  85, 105, 125, 145, 165, 185])

In [21]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [22]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [23]:
# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)

n_features = X.shape[2]

print(X.shape, y.shape,'\n')
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

(7, 3, 2) (7,) 

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


We can see that the X component has a three-dimensional structure.

The first dimension is the number of samples, in this case 7. The second dimension is the number of time steps per sample, in this case 3, the value specified to the function. Finally, the last dimension specifies the number of parallel time series or the number of variables, in this case 2 for the two parallel series.

This is the exact three-dimensional structure expected by an LSTM as input. The data is ready to use without further reshaping.

We can then see that the input and output for each sample is printed, showing the three time steps for each of the two input series and the associated output for each sample.

In [24]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=200, verbose=0)

x_input = np.array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
model.predict(x_input, verbose=0)

array([[209.71779]], dtype=float32)

Let's try this with Apple's data